In [1]:
!PYTHON -V


/bin/bash: line 1: PYTHON: command not found


In [2]:
!python -V

Python 3.12.4


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
import pandas as pd

def read_dataframe(filename):
    # Reading the Parquet file instead of CSV
    df = pd.read_parquet(filename)
    
    # Converting columns to datetime
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    # Calculating duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filtering the duration between 1 and 60 minutes
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Converting categorical columns to string
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


In [10]:
# Reading train and validation dataframes
df_train = read_dataframe('data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('data/green_tripdata_2024-02.parquet')

# Printing the head of the train dataframe
print(df_train.head())

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2024-01-01 00:46:55   2024-01-01 00:58:25                  N   
1         2  2024-01-01 00:31:42   2024-01-01 00:52:34                  N   
2         2  2024-01-01 00:30:21   2024-01-01 00:49:23                  N   
3         1  2024-01-01 00:30:20   2024-01-01 00:42:12                  N   
4         2  2024-01-01 00:32:38   2024-01-01 00:43:37                  N   

   RatecodeID PULocationID DOLocationID  passenger_count  trip_distance  \
0         1.0          236          239              1.0           1.98   
1         1.0           65          170              5.0           6.54   
2         1.0           74          262              1.0           3.08   
3         1.0           74          116              1.0           2.40   
4         1.0           74          243              1.0           5.14   

   fare_amount  ...  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0         12.8  .

In [11]:
len(df_train), len(df_val)

(54373, 51497)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [23]:
X_train

<54373x5093 sparse matrix of type '<class 'numpy.float64'>'
	with 108746 stored elements in Compressed Sparse Row format>

In [24]:
train_dicts

[{'PU_DO': '236_239', 'trip_distance': 1.98},
 {'PU_DO': '65_170', 'trip_distance': 6.54},
 {'PU_DO': '74_262', 'trip_distance': 3.08},
 {'PU_DO': '74_116', 'trip_distance': 2.4},
 {'PU_DO': '74_243', 'trip_distance': 5.14},
 {'PU_DO': '33_209', 'trip_distance': 2.0},
 {'PU_DO': '74_238', 'trip_distance': 3.2},
 {'PU_DO': '166_239', 'trip_distance': 2.01},
 {'PU_DO': '226_226', 'trip_distance': 0.31},
 {'PU_DO': '7_129', 'trip_distance': 2.32},
 {'PU_DO': '42_75', 'trip_distance': 2.69},
 {'PU_DO': '41_141', 'trip_distance': 3.73},
 {'PU_DO': '130_196', 'trip_distance': 5.06},
 {'PU_DO': '74_69', 'trip_distance': 2.93},
 {'PU_DO': '41_74', 'trip_distance': 1.26},
 {'PU_DO': '55_210', 'trip_distance': 4.09},
 {'PU_DO': '41_42', 'trip_distance': 1.66},
 {'PU_DO': '42_151', 'trip_distance': 1.55},
 {'PU_DO': '41_42', 'trip_distance': 1.78},
 {'PU_DO': '255_255', 'trip_distance': 0.26},
 {'PU_DO': '41_24', 'trip_distance': 1.49},
 {'PU_DO': '255_79', 'trip_distance': 3.9},
 {'PU_DO': '74_7

In [28]:
dv.feature_names_

['PU_DO=101_131',
 'PU_DO=101_135',
 'PU_DO=101_188',
 'PU_DO=101_56',
 'PU_DO=101_82',
 'PU_DO=102_10',
 'PU_DO=102_140',
 'PU_DO=102_150',
 'PU_DO=102_196',
 'PU_DO=102_205',
 'PU_DO=102_233',
 'PU_DO=102_76',
 'PU_DO=102_82',
 'PU_DO=102_83',
 'PU_DO=102_95',
 'PU_DO=106_181',
 'PU_DO=106_188',
 'PU_DO=106_189',
 'PU_DO=106_226',
 'PU_DO=106_228',
 'PU_DO=106_231',
 'PU_DO=106_25',
 'PU_DO=106_263',
 'PU_DO=106_40',
 'PU_DO=106_56',
 'PU_DO=106_61',
 'PU_DO=106_68',
 'PU_DO=106_82',
 'PU_DO=106_97',
 'PU_DO=108_123',
 'PU_DO=108_149',
 'PU_DO=108_210',
 'PU_DO=108_228',
 'PU_DO=108_257',
 'PU_DO=108_29',
 'PU_DO=108_4',
 'PU_DO=108_52',
 'PU_DO=108_55',
 'PU_DO=108_67',
 'PU_DO=108_72',
 'PU_DO=108_91',
 'PU_DO=10_10',
 'PU_DO=10_122',
 'PU_DO=10_130',
 'PU_DO=10_132',
 'PU_DO=10_134',
 'PU_DO=10_135',
 'PU_DO=10_17',
 'PU_DO=10_181',
 'PU_DO=10_191',
 'PU_DO=10_205',
 'PU_DO=10_220',
 'PU_DO=10_258',
 'PU_DO=10_62',
 'PU_DO=10_76',
 'PU_DO=10_89',
 'PU_DO=112_107',
 'PU_DO=112_112'

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/ubuntu/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.995211090245985